# Quickstart: Deploy Azure IoT Operations to an Arc-enabled Kubernetes cluster using Codespaces and AKS EE

<div class="alert alert-block alert-info">
<b>IMPORTANT:</b> Azure IoT Operations Preview – enabled by Azure Arc is currently in PREVIEW.
You shouldn't use this preview software in production environments.

See the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/) for legal terms that apply to Azure features that are in beta, preview, or otherwise not yet released into general availability.</div>

In this quickstart, you will deploy a suite of IoT services to an Azure Arc-enabled Kubernetes cluster so that you can remotely manage your devices and workloads. Azure IoT Operations Preview is a digital operations suite of services that includes Azure IoT Orchestrator. This quickstart guides you through using Orchestrator to deploy these services to a Kubernetes cluster. At the end of the quickstart, you have a cluster that you can manage from the cloud that's generating sample data to use in the following quickstarts.

The services deployed in this quickstart include:

* [Azure IoT Orchestrator](../deploy-custom/overview-orchestrator.md)
* [Azure IoT MQ](../manage-mqtt-connectivity/overview-iot-mq.md)
* [Azure IoT OPC UA broker](../manage-devices-assets/overview-opcua-broker.md) with simulated thermostat asset to start generating data
* [Azure IoT Data Processor](../process-data/overview-data-processor.md) with a demo pipeline to start routing the simulated data
* [Azure IoT Akri](../manage-devices-assets/overview-akri.md)
* [Azure Device Registry](../manage-devices-assets/overview-manage-assets.md#manage-assets-as-azure-resources-in-a-centralized-registry)
* [Azure IoT Layered Network Management](../manage-layered-network/overview-layered-network.md)
* [Observability](../monitor/howto-configure-observability.md)

## Prerequisites
To prepare your Azure Arc-enabled Kubernetes cluster, you need:

- An Azure subscription. If you don't have an Azure subscription, create a [free account](https://azure.microsoft.com/free/?WT.mc_id=A261C142F) before you begin.
- [Azure CLI version 2.42.0 or newer installed](/cli/azure/install-azure-cli) on your development machine.
- Hardware that meets the [system requirements](/azure/azure-arc/kubernetes/system-requirements).

### Create a cluster

This section provides steps to prepare and Arc-enable clusters in validated environments on Linux and Windows as well as GitHub Codespaces in the cloud.

[Azure Kubernetes Service Edge Essentials](/azure/aks/hybrid/aks-edge-overview) is an on-premises Kubernetes implementation of Azure Kubernetes Service (AKS) that automates running containerized applications at scale. AKS Edge Essentials includes a Microsoft-supported Kubernetes platform that includes a lightweight Kubernetes distribution with a small footprint and simple installation experience, making it easy for you to deploy Kubernetes on PC-class or "light" edge hardware.

1. Sign in to Azure CLI on your development machine or in your codespace terminal. To prevent potential permission issues later, sign in interactively with a browser here even if you've already logged in before.

```azurecli
az login
```

> [!NOTE]
> When using a Github codespace in a browser, `az login` returns a localhost error in the browser window after logging in. To fix, either:
>
> * Open the codespace in VS Code desktop, and then run `az login` again in the browser terminal.
> * After you get the localhost error on the browser, copy the URL from the browser and run `curl "<URL>"` in a new terminal tab. You should see a JSON response with the message "You have logged into Microsoft Azure!."

1. Set environment variables for the rest of the setup. Replace values in `<>` with valid values or names of your choice. A new cluster and resource group are created in your Azure subscription based on the names you provide:

In [ ]:
# Azure region where the created resource group will be located
# Currently supported regions: "eastus", "eastus2", "westus", "westus2", "westus3", "westeurope", or "northeurope"
$LOCATION="WestUS3"

$RESOURCE_GROUP = Read-Host -Prompt "Name of a new resource group to create which will hold the Arc-enabled cluster and Alice Springs resources"
Write-Host "Resource group: $RESOURCE_GROUP" -ForegroundColor Blue

$CLUSTER_NAME = Read-Host -Prompt "Name of the Arc-enabled cluster to create in your resource group"
Write-Host "Arc cluster name: $CLUSTER_NAME" -ForegroundColor Blue

1. Set the Azure subscription context for all commands:

In [ ]:
# Id of the subscription where your resource group and Arc-enabled cluster will be created

$SUBSCRIPTION_ID = (az account show --query id -o tsv)
$TENANT_ID = (az account show --query tenantId -o tsv)
az account set -s $SUBSCRIPTION_ID

>[!TIP]
>You can use the [AksEdgeQuickStartForAio.ps1](https://github.com/Azure/AKS-Edge/blob/main/tools/scripts/AksEdgeQuickStart/AksEdgeQuickStartForAio.ps1) script to automate the steps in this section and connect your cluster.

In an elevated PowerShell window, run the following commands:

In [ ]:
$url = "https://raw.githubusercontent.com/Azure/AKS-Edge/main/tools/scripts/AksEdgeQuickStart/AksEdgeQuickStartForAio.ps1"
Invoke-WebRequest -Uri $url -OutFile .\AksEdgeQuickStartForAio.ps1
Unblock-File .\AksEdgeQuickStartForAio.ps1
Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process -Force
.\AksEdgeQuickStartForAio.ps1 -SubscriptionId $SUBSCRIPTION_ID -TenantId $TENANT_ID -ResourceGroupName $RESOURCE_GROUP  -Location $LOCATION  -ClusterName $CLUSTER_NAME

>Your machine might reboot as part of this process. If so, run the whole set of commands again.

Prepare your machine for AKS Edge Essentials.

1. Download the [installer for the validated AKS Edge Essentials](https://aka.ms/aks-edge/msi-k3s-1.2.414.0) version to your local machine.

1. Complete the steps in [Prepare your machine for AKS Edge Essentials](/azure/aks/hybrid/aks-edge-howto-setup-machine). Be sure to use the validated installer you downloaded in the previous step and not the most recent version.

Set up an AKS Edge Essentials cluster on your machine.

1. Complete the steps in [Create a single machine deployment](/azure/aks/hybrid/aks-edge-howto-single-node-deployment).

    At the end of [Step 1: single machine configuration parameters](/azure/aks/hybrid/aks-edge-howto-single-node-deployment#step-1-single-machine-configuration-parameters), modify the following values in the _aksedge-config.json_ file as follows:

    ```json
    `Init.ServiceIPRangeSize` = 10
    `LinuxNode.DataSizeInGB` = 30
    `LinuxNode.MemoryInMB` = 8192
    ```

1. Install **local-path** storage in the cluster by running the following command:

In [ ]:
kubectl apply -f https://raw.githubusercontent.com/Azure/AKS-Edge/main/samples/storage/local-path-provisioner/local-path-storage.yaml

---
## Arc-enable your cluster

To connect your cluster to Azure Arc:

1. Register the required resource providers in your subscription:

In [ ]:
az provider register -n "Microsoft.ExtendedLocation"
az provider register -n "Microsoft.Kubernetes"
az provider register -n "Microsoft.KubernetesConfiguration"
az provider register -n "Microsoft.IoTOperationsOrchestrator"
az provider register -n "Microsoft.IoTOperationsMQ"
az provider register -n "Microsoft.IoTOperationsDataProcessor"
az provider register -n "Microsoft.DeviceRegistry"

1. Use the [az group create](/cli/azure/group#az-group-create) command to create a resource group in your Azure subscription to store all the resources:

In [ ]:
az group create --location $LOCATION --resource-group $RESOURCE_GROUP --subscription $SUBSCRIPTION_ID

1. Use the [az connectedk8s connect](/cli/azure/connectedk8s#az-connectedk8s-connect) command to Arc-enable your Kubernetes cluster and manage it in the resource group you created in the previous step:

In [ ]:
az connectedk8s connect -n $CLUSTER_NAME -l $LOCATION -g $RESOURCE_GROUP --subscription $SUBSCRIPTION_ID

1. Fetch the `objectId` or `id` of the Microsoft Entra ID application that the Azure Arc service uses.

In [ ]:
$OBJECT_ID = (az ad sp show --id bc313c14-388c-4e7d-a58e-70017303ee3b --query id -o tsv)

1. Use the [az connectedk8s enable-features](/cli/azure/connectedk8s#az-connectedk8s-enable-features) command to enable custom location support on your cluster. Use the `objectId` or `id` value from the previous command to enable custom locations on the cluster:

In [ ]:
az connectedk8s enable-features -n $CLUSTER_NAME -g $RESOURCE_GROUP --custom-locations-oid <objectId/id> --features cluster-connect custom-locations

## Verify your cluster

To verify that your Kubernetes cluster is now Azure Arc-enabled, run the following command:

In [ ]:
kubectl get deployments,pods -n azure-arc

---

## Configure cluster and deploy Azure IoT Operations

Part of the deployment process is to configure your cluster so that it can communicate securely with your Azure IoT Operations components and key vault. The Azure CLI command `az iot ops init` does this for you. Once your cluster is configured, then you can deploy Azure IoT Operations.

Use the Azure CLI to create a key vault, build the `az iot ops init` command based on your resources, and then deploy Azure IoT Operations components to your Arc-enabled Kubernetes cluster.

### Create a key vault

You can use an existing key vault for your secrets, but verify that the **Permission model** is set to **Vault access policy**. You can check this setting in the Azure portal in the **Access configuration** section of an existing key vault. Or use the [az keyvault show](/cli/azure/keyvault#az-keyvault-show) command to check that `enableRbacAuthorization` is false.

To create a new key vault, use the following command:

In [ ]:
$KEY_VAULT_NAME = Read-Host -Prompt "Name of the key vault to create in your resource group"

az keyvault create --enable-rbac-authorization false --name $KEY_VAULT_NAME --resource-group $RESOURCE_GROUP


### Deploy Azure IoT Operations

1. In the Azure portal search bar, search for and select **Azure Arc**.

1. Select **Azure IoT Operations (preview)** from the **Application services** section of the Azure Arc menu.

   :::image type="content" source="./media/quickstart-deploy/arc-iot-operations.png" alt-text="Screenshot of selecting Azure IoT Operations from Azure Arc.":::

1. Select **Create**.

1. On the **Basics** tab of the **Install Azure IoT Operations Arc Extension** page, provide the following information:

   | Field | Value |
   | ----- | ----- |
   | **Subscription** | Select the subscription that contains your Arc-enabled Kubernetes cluster. |
   | **Resource group** | Select the resource group that contains your Arc-enabled Kubernetes cluster. |
   | **Cluster name** | Select your cluster. When you do, the **Custom location** and **Deployment details** sections autofill. |

   :::image type="content" source="./media/quickstart-deploy/install-extension-basics.png" alt-text="Screenshot of the basics tab for installing the Azure IoT Operations Arc extension in the Azure portal.":::

1. Select **Next: Configuration**.

1. On the **Configuration** tab, provide the following information:

   | Field | Value |
   | ----- | ----- |
   | **Deploy a simulated PLC** | Switch this toggle to **Yes**. The simulated PLC creates demo telemetry data that you use in the following quickstarts. |
   | **Mode** | Set the MQ configuration mode to **Auto**. |

   :::image type="content" source="./media/quickstart-deploy/install-extension-configuration.png" alt-text="Screenshot of the configuration tab for installing the Azure IoT Operations Arc extension in the Azure portal.":::

1. Select **Next: Automation**.

1. On the **Automation** tab, select **Pick or create an Azure Key Vault**.

   :::image type="content" source="./media/quickstart-deploy/install-extension-automation-1.png" alt-text="Screenshot of selecting your key vault in the automation tab for installing the Azure IoT Operations Arc extension in the Azure portal.":::

1. Provide the following information to connect a key vault:

   | Field | Value |
   | ----- | ----- |
   | **Subscription** | Select the subscription that contains your Arc-enabled Kubernetes cluster. |
   | **Key vault** | Choose the key vault that you created in the previous section from the drop-down list. |

1. Select **Select**.

1. On the **Automation** tab, the automation commands are populated based on your chosen cluster and key vault. Copy the **Required** CLI command.

   :::image type="content" source="./media/quickstart-deploy/install-extension-automation-2.png" alt-text="Screenshot of copying the CLI command from the automation tab for installing the Azure IoT Operations Arc extension in the Azure portal.":::

1. Run the `az iot ops init` command on your development machine or in your codespace terminal.

   >[!TIP]
   >If you get an error that says *Your device is required to be managed to access your resource*, go back to the previous step and make sure that you signed in interactively.



In [ ]:
# Remove old extensions:
az extension remove --name azure-iot-ops
az extension remove --name azure-edge

# Install extension "0.1.0b1"
az extension add --source $(curl -w "%{url_effective}\n" -I -L -s -S https://aka.ms/aziotopscli-latest -o /dev/null) -y

In [ ]:
az iot ops init

1. Return to the Azure portal and select **Review + Create**.

1. Wait for the validation to pass and then select **Create**.

## View resources in your cluster

While the deployment is in progress, you can watch the resources being applied to your cluster. You can use kubectl commands to observe changes on the cluster or, since the cluster is Arc-enabled, you can use the Azure portal.

To view the pods on your cluster, run the following command:

In [ ]:
#!pwsh
kubectl get pods -n azure-iot-operations


It can take several minutes for the deployment to complete. Continue running the `get pods` command to refresh your view.

To view your cluster on the Azure portal, use the following steps:

1. In the Azure portal, navigate to the resource group that contains your cluster.

1. From the **Overview** of the resource group, select the name of your cluster.

1. On your cluster, select **Extensions** from the menu.

   You can see that your cluster is running extensions of the type **microsoft.iotoperations.x**, which is the group name for all of the Azure IoT Operations components and the orchestration service.

   There's also an extension called **akvsecretsprovider**. This extension is the secrets provider that you configured and installed on your cluster with the `az iot ops init` command. You might delete and reinstall the Azure IoT Operations components during testing, but keep the secrets provider extension on your cluster.

## How did we solve the problem?

In this quickstart, you configured your Arc-enabled Kubernetes cluster so that it could communicate securely with your Azure IoT Operations components. Then, you deployed those components to your cluster. For this test scenario, you have a single Kubernetes cluster that's probably running locally on your machine. In a production scenario, however, you can use the same steps to deploy workloads to many clusters across many sites.

## Clean up resources

If you're continuing on to the next quickstart, keep all of your resources.

If you want to delete the Azure IoT Operations deployment but plan on reinstalling it on your cluster, be sure to keep the secrets provider on your cluster. In your cluster on the Azure portal, select the extensions of the type **microsoft.iotoperations.x** and **microsoft.deviceregistry.assets**, then select **Uninstall**.

If you want to delete all of the resources you created for this quickstart, delete the Kubernetes cluster that you deployed Azure IoT Operations to and remove the Azure resource group that contained the cluster.

## Next steps

> [!div class="nextstepaction"]
> [Quickstart: Add OPC UA assets to your Azure IoT Operations cluster](quickstart-add-assets.md)